## Serve Placement Visual

In [2]:
import pandas as pd
import numpy as np
import os

player = "Kaylan Bigun"
path = "Shot_Visuals_KaylanBigun_Combined.csv"

output_dir = os.path.join(os.getcwd())
os.makedirs(output_dir, exist_ok=True)

In [3]:


events = pd.read_csv(path)
events['pointWonBy'] = events.groupby(['pointNumber', 'player2Name'])['pointWonBy'].bfill()

# Filter and mutate serves
serves = events.groupby(['pointNumber', 'player2Name']).apply(lambda df: pd.Series({
    'pointNumber': df['pointNumber'].iloc[0],
    'pointStartTime': df['pointStartTime'].iloc[0],
    'serverName': df['serverName'].iloc[0],
    'x': df['firstServeXCoord'].iloc[0] if df['firstServeIn'].iloc[0] == 1 else df['secondServeXCoord'].iloc[0],
    'y': df['firstServeYCoord'].iloc[0] if df['firstServeIn'].iloc[0] == 1 else df['secondServeYCoord'].iloc[0],
    'serveIn': (df['firstServeIn'].iloc[0] == 1) | (df['secondServeIn'].iloc[0] == 1),
    'side': df['side'].iloc[0],
    'serveInPlacement': df['serveInPlacement'].iloc[0],
    'pointWonByUCLA': (df['pointWonBy'].iloc[0] == player),
    'isAce': df['isAce'].iloc[0]
})).reset_index(drop=True)

# serves = serves[serves['serveIn']]
serves.shape[0]
serves_ucla = serves[(serves['serverName'] == player) & (serves['serveIn'])].copy()
serves_ucla.shape[0]

192

In [4]:
# Load the data
events = pd.read_csv(path)
events['pointWonBy'] = events.groupby(['pointNumber', 'player2Name'])['pointWonBy'].bfill()

# Filter and mutate serves
serves = events.groupby(['pointNumber', 'player2Name']).apply(lambda df: pd.Series({
    'pointNumber': df['pointNumber'].iloc[0],
    'pointStartTime': df['pointStartTime'].iloc[0],
    'serverName': df['serverName'].iloc[0],
    'x': df['firstServeXCoord'].iloc[0] if df['firstServeIn'].iloc[0] == 1.0 else df['secondServeXCoord'].iloc[0],
    'y': df['firstServeYCoord'].iloc[0] if df['firstServeIn'].iloc[0] == 1.0 else df['secondServeYCoord'].iloc[0],
    'serveIn': (df['firstServeIn'].iloc[0] == 1.0) or (df['secondServeIn'].iloc[0] == 1.0),
    'side': df['side'].iloc[0],
    'serveInPlacement': df['serveInPlacement'].iloc[0],
    'pointWonByUCLA': (df['pointWonBy'].iloc[0] == player),
    'isAce': df['isAce'].iloc[0]
})).reset_index(drop=True)

serves = serves[serves['serveIn']]

# Filter serves where server is UCLA player and serve was in
serves_ucla = serves[(serves['serverName'] == player) & (serves['serveIn'])].copy()

# Modify the coordinates based on the y-value
serves_ucla['x'] = np.where(serves_ucla['y'] < 0, -serves_ucla['x'], serves_ucla['x'])
serves_ucla['y'] = np.where(serves_ucla['y'] < 0, -serves_ucla['y'], serves_ucla['y'])

# Define the serve outcome based on conditions
serves_ucla['serveOutcome'] = np.where(
    serves_ucla['isAce'] == 1.0, 'Ace',
    np.where(serves_ucla['pointWonByUCLA'], 'Won', 'Lost')
)

# Cleaning for valid serve placements.
valid_placements = ['Wide', 'T', 'Body']
serves_ucla = serves_ucla[serves_ucla['serveInPlacement'].isin(valid_placements)]

# Group by side and serveInPlacement, and calculate count and serves won
distribution = serves_ucla.groupby(['side', 'serveInPlacement']).agg(
    count=('pointNumber', 'size'),
    serves_won=('pointWonByUCLA', 'sum')
).reset_index() 

# Calculate the win percentage (proportion)
distribution['proportion'] = distribution['serves_won'] / distribution['count']

# Find the minimum and maximum proportions
min_proportion = distribution['proportion'].min()
max_proportion = distribution['proportion'].max()

# Create labels DataFrame and determine if each value is max, min, or neither
labels = distribution.copy()
labels['proportion_label'] = (labels['proportion'] * 100).round(1).astype(str) + "%"
labels['count_label'] = labels['count']

# Add the x positions based on side and serveInPlacement
labels['x'] = np.where(
    (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Wide'), 131.25,
    np.where(
        (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Body'), 78.75,
        np.where(
            (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'T'), 26.25,
            np.where(
                (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'T'), -26.25,
                np.where(
                    (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Body'), -78.75,
                    np.where(
                        (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Wide'), -131.25,
                        np.nan
                    )
                )
            )
        )
    )
)

# Determine text color and max/min status
labels['text_color'] = np.where(
    labels['proportion'] == min_proportion, "darkred",
    np.where(labels['proportion'] == max_proportion, "darkgreen", "black")
)

labels['max_min'] = np.where(
    labels['proportion'] == max_proportion, "max",
    np.where(labels['proportion'] == min_proportion, "min", "no")
)

# Export data frames as JSON
serve_place_json = serves_ucla.to_json(orient='records')
serve_place_labels_json = labels.to_json(orient='records')

with open(os.path.join(output_dir,'serve_place.json'), 'w') as f:
    f.write(serve_place_json)

with open(os.path.join(output_dir,'serve_place_labels.json'), 'w') as f:
    f.write(serve_place_labels_json)